In [ ]:
pip install --upgrade cptac

In [28]:
import pandas as pd
import sys
import os
import cptac
from xml.dom import minidom

In [15]:
def fixNewLines(s):
    if "\n" in s:
        nList = s.split("\n")
    else:
        nList = []
        nList.append(s)
    return nList

In [16]:
def fixTabs(s):
    if "\t" in s:
        tList = s.split("\t")
    else:
        tList = []
        tList.append(s)
    return tList

In [17]:
def fixParen(q):
    fixedList = []
    list1 = q.split(")")
    for i in list1:
        i = i.strip()
        list2 = i.split("(")
        for j in list2:
            j = j.strip()
            if j != "":
                fixedList.append(j)
    return fixedList

In [18]:
#function to split each item with a newline into two items

#fix if genes are in parenthesis

def fixParsingErrors(interactingList):
    fixedList = []
    for i in interactingList:
        i = i.strip()
        noSpaces = i.split(" ")
        for item in noSpaces:
            newlineFreeList = fixNewLines(item)
            for n in newlineFreeList:
                tabFreeList = fixTabs(n)
                for t in tabFreeList:
                    noParen = fixParen(t)
                    fixedList.extend(noParen)
    return list(set(fixedList))

In [19]:
def getUniprotGeneList():
    filePath = '/Users/coribushman/github/cptac/cptac/utils/Uniprot_Proteome.tsv'
    bioplex_interactions = pd.read_csv(filePath, sep='\t')
    uniprotList = []
    for ind, row in bioplex_interactions.iterrows():
        prots = str(row['Gene names'])
        #print(prots)
        prot_arr = prots.split(' ')
        #print(prot_arr)
        for prot in prot_arr:
            if prot not in uniprotList:
                uniprotList.append(prot)
    return uniprotList

In [20]:
def intersectWithUniprot(interactingGenes):
    #get uniprot genes from file
    uniprotGenes = getUniprotGeneList()
    
    #intersect
    geneSet = set(interactingGenes)
    uniprotSet = set(uniprotGenes)   #convert to set
    intersectSet = geneSet.intersection(uniprotSet)
    
    #today = date.today()
    #todayDate = today.strftime("%m/%d/%y")
    
    #print("WikiPathways Interacting Proteins as of " + str(todayDate))  #change from date to version of download
    return list(intersectSet)

In [21]:
def getGenesPerPathwayDict():
    path = '/Users/coribushman/Desktop/wikipathways-20191010-gpml-Homo_sapiens'

    pathwaysPerGene = {}
    genesPerPathway = {}
    #print("making dict")
    for fileName in os.listdir(path):
        if fileName == ".DS_Store":
            continue
        filePath = path + "/" + fileName
        pathwayGenes = []

        xmldoc = minidom.parse(filePath)
        pathway = xmldoc.getElementsByTagName("Pathway")[0]
        pathwayName = pathway.getAttribute("Name")

        dataNodes = pathway.getElementsByTagName("DataNode")
        for node in dataNodes:
            if node.getAttribute("Type") == "GeneProduct":
                geneName = node.getAttribute("TextLabel")
                pathwayGenes.append(geneName)

        pathwayGenes = fixParsingErrors(pathwayGenes)
        #print(pathwayName + " errors fixed")
        pathwayGenes = intersectWithUniprot(pathwayGenes)
        #print(pathwayName + " filtered through uniprot")
        #pathwayGenes = intersectWithUniprot(pathwayGenes)
        genesPerPathway[pathwayName] = pathwayGenes
    return genesPerPathway

In [22]:
def getAllPathways(genesPerPathway):
    allPathways = genesPerPathway.keys()
    allPathways = list(allPathways)
    return allPathways

In [23]:
def getAllGenes(genesPerPathway):
    allGenes = []
    for geneList in genesPerPathway.values():
        allGenes.extend(geneList)
    allGenes = list(set(allGenes))
    allGenes = fixParsingErrors(allGenes)
    return allGenes

In [24]:
#gene, list of genes, pathways, list of pathways
#have people test it to make sure it works

#filter dict genes (they have newlines), intersect w/ uniprot

In [29]:
genesPerPathway = getGenesPerPathwayDict()
allPathways = getAllPathways(genesPerPathway)
allGenes = getAllGenes(genesPerPathway)

making dict
Target Of Rapamycin (TOR) Signaling errors fixed
Target Of Rapamycin (TOR) Signaling filtered through uniprot
NRF2-ARE regulation errors fixed
NRF2-ARE regulation filtered through uniprot
PDGF Pathway errors fixed
PDGF Pathway filtered through uniprot
Eukaryotic Transcription Initiation errors fixed
Eukaryotic Transcription Initiation filtered through uniprot
Interleukin-11 Signaling Pathway errors fixed
Interleukin-11 Signaling Pathway filtered through uniprot
Vitamin D Receptor Pathway errors fixed
Vitamin D Receptor Pathway filtered through uniprot
T-Cell antigen Receptor (TCR) pathway during Staphylococcus aureus infection errors fixed
T-Cell antigen Receptor (TCR) pathway during Staphylococcus aureus infection filtered through uniprot
Phosphatidylcholine catabolism errors fixed
Phosphatidylcholine catabolism filtered through uniprot
SRF and miRs in Smooth Muscle Differentiation and Proliferation errors fixed
SRF and miRs in Smooth Muscle Differentiation and Proliferati

Oxidation by Cytochrome P450 filtered through uniprot
Fragile X Syndrome  errors fixed
Fragile X Syndrome  filtered through uniprot
Hematopoietic Stem Cell Gene Regulation by GABP alpha/beta Complex errors fixed
Hematopoietic Stem Cell Gene Regulation by GABP alpha/beta Complex filtered through uniprot
G1 to S cell cycle control errors fixed
G1 to S cell cycle control filtered through uniprot
Mammary gland development pathway - Pregnancy and lactation (Stage 3 of 4) errors fixed
Mammary gland development pathway - Pregnancy and lactation (Stage 3 of 4) filtered through uniprot
miRNA regulation of p53 pathway in prostate cancer errors fixed
miRNA regulation of p53 pathway in prostate cancer filtered through uniprot
Insulin signalling in human adipocytes (normal condition) errors fixed
Insulin signalling in human adipocytes (normal condition) filtered through uniprot
Brain-Derived Neurotrophic Factor (BDNF) signaling pathway errors fixed
Brain-Derived Neurotrophic Factor (BDNF) signaling

RANKL/RANK (Receptor activator of NFKB (ligand)) Signaling Pathway filtered through uniprot
Ectoderm Differentiation errors fixed
Ectoderm Differentiation filtered through uniprot
G13 Signaling Pathway errors fixed
G13 Signaling Pathway filtered through uniprot
Photodynamic therapy-induced NFE2L2 (NRF2) survival signaling errors fixed
Photodynamic therapy-induced NFE2L2 (NRF2) survival signaling filtered through uniprot
DNA Damage Response errors fixed
DNA Damage Response filtered through uniprot
TFs Regulate miRNAs related to cardiac hypertrophy errors fixed
TFs Regulate miRNAs related to cardiac hypertrophy filtered through uniprot
Nanoparticle-mediated activation of receptor signaling errors fixed
Nanoparticle-mediated activation of receptor signaling filtered through uniprot
Phosphodiesterases in neuronal function errors fixed
Phosphodiesterases in neuronal function filtered through uniprot
Amino acid conjugation errors fixed
Amino acid conjugation filtered through uniprot
Biochemi

Lamin A-processing pathway filtered through uniprot
Cerebral Organic Acidurias, including diseases errors fixed
Cerebral Organic Acidurias, including diseases filtered through uniprot
ESC Pluripotency Pathways errors fixed
ESC Pluripotency Pathways filtered through uniprot
Cardiac Progenitor Differentiation errors fixed
Cardiac Progenitor Differentiation filtered through uniprot
RalA downstream regulated genes errors fixed
RalA downstream regulated genes filtered through uniprot
Monoamine Transport errors fixed
Monoamine Transport filtered through uniprot
B Cell Receptor Signaling Pathway errors fixed
B Cell Receptor Signaling Pathway filtered through uniprot
Synthesis and Degradation of Ketone Bodies errors fixed
Synthesis and Degradation of Ketone Bodies filtered through uniprot
G Protein Signaling Pathways errors fixed
G Protein Signaling Pathways filtered through uniprot
miRNAs involved in DNA damage response errors fixed
miRNAs involved in DNA damage response filtered through unip

PTF1A related regulatory pathway filtered through uniprot
Fas Ligand (FasL) pathway and Stress induction of Heat Shock Proteins (HSP) regulation errors fixed
Fas Ligand (FasL) pathway and Stress induction of Heat Shock Proteins (HSP) regulation filtered through uniprot
Role Altered Glycolysation of MUC1 in Tumour Microenvironment errors fixed
Role Altered Glycolysation of MUC1 in Tumour Microenvironment filtered through uniprot
Complement and Coagulation Cascades errors fixed
Complement and Coagulation Cascades filtered through uniprot
Hfe effect on hepcidin production errors fixed
Hfe effect on hepcidin production filtered through uniprot
Tyrosine Metabolism errors fixed
Tyrosine Metabolism filtered through uniprot
Tamoxifen metabolism errors fixed
Tamoxifen metabolism filtered through uniprot
Development and heterogeneity of the ILC family errors fixed
Development and heterogeneity of the ILC family filtered through uniprot
Gene regulatory network modelling somitogenesis  errors fixe

Peroxisomal beta-oxidation of tetracosanoyl-CoA filtered through uniprot
GPCRs, Class C Metabotropic glutamate, pheromone errors fixed
GPCRs, Class C Metabotropic glutamate, pheromone filtered through uniprot
Adipogenesis errors fixed
Adipogenesis filtered through uniprot
Carnosine's role in muscle contraction errors fixed
Carnosine's role in muscle contraction filtered through uniprot
Pathways Affected in Adenoid Cystic Carcinoma errors fixed
Pathways Affected in Adenoid Cystic Carcinoma filtered through uniprot
Wnt Signaling Pathway and Pluripotency errors fixed
Wnt Signaling Pathway and Pluripotency filtered through uniprot
Fatty Acid Beta Oxidation errors fixed
Fatty Acid Beta Oxidation filtered through uniprot
Dopaminergic Neurogenesis errors fixed
Dopaminergic Neurogenesis filtered through uniprot
Bone Morphogenic Protein (BMP) Signalling and Regulation errors fixed
Bone Morphogenic Protein (BMP) Signalling and Regulation filtered through uniprot
Caffeine and Theobromine metaboli

Gastrin Signaling Pathway filtered through uniprot
Matrix Metalloproteinases errors fixed
Matrix Metalloproteinases filtered through uniprot
Aflatoxin B1 metabolism errors fixed
Aflatoxin B1 metabolism filtered through uniprot
Toll-like Receptor Signaling errors fixed
Toll-like Receptor Signaling filtered through uniprot
Microglia Pathogen Phagocytosis Pathway errors fixed
Microglia Pathogen Phagocytosis Pathway filtered through uniprot
Joubert Syndrome errors fixed
Joubert Syndrome filtered through uniprot
Pathways Regulating Hippo Signaling errors fixed
Pathways Regulating Hippo Signaling filtered through uniprot
MTHFR deficiency errors fixed
MTHFR deficiency filtered through uniprot
Splicing factor NOVA regulated synaptic proteins errors fixed
Splicing factor NOVA regulated synaptic proteins filtered through uniprot
Translation Factors errors fixed
Translation Factors filtered through uniprot
Metabolic reprogramming in colon cancer errors fixed
Metabolic reprogramming in colon cance

Glycosylation and related congenital defects filtered through uniprot
Modified nucleosides derived from t-RNA as urinary cancer markers errors fixed
Modified nucleosides derived from t-RNA as urinary cancer markers filtered through uniprot
miR-517 relationship with ARCN1 and USP1 errors fixed
miR-517 relationship with ARCN1 and USP1 filtered through uniprot
The alternative pathway of fetal androgen synthesis errors fixed
The alternative pathway of fetal androgen synthesis filtered through uniprot
MAPK Cascade errors fixed
MAPK Cascade filtered through uniprot
Selenium Micronutrient Network errors fixed
Selenium Micronutrient Network filtered through uniprot
Neovascularisation processes errors fixed
Neovascularisation processes filtered through uniprot
Wnt/beta-catenin Signaling Pathway in Leukemia errors fixed
Wnt/beta-catenin Signaling Pathway in Leukemia filtered through uniprot
The effect of progerin on the involved genes in Hutchinson-Gilford Progeria Syndrome errors fixed
The effe

In [30]:
#df = pd.DataFrame(genesPerPathway, columns = allPathways, index = allGenes)
df = pd.DataFrame(columns = allPathways, index = allGenes)
df = df.fillna(False)


for pathway in df.columns:
    for gene in df.index:
        if gene in genesPerPathway[pathway]:
            #df.replace(to_replace = False, value = True)
            df.at[gene, pathway] = True
            
df

,Target Of Rapamycin (TOR) Signaling,NRF2-ARE regulation,PDGF Pathway,Eukaryotic Transcription Initiation,Interleukin-11 Signaling Pathway,Vitamin D Receptor Pathway,T-Cell antigen Receptor (TCR) pathway during Staphylococcus aureus infection,Phosphatidylcholine catabolism,SRF and miRs in Smooth Muscle Differentiation and Proliferation,Glucocorticoid and Mineralcorticoid Metabolism,...,Serotonin Transporter Activity,TCA Cycle (aka Krebs or citric acid cycle),Serotonin and anxiety,Sudden Infant Death Syndrome (SIDS) Susceptibility Pathways,Canonical NF-KB pathway,Photodynamic therapy-induced HIF-1 survival signaling,Arrhythmogenic Right Ventricular Cardiomyopathy,Ethanol metabolism resulting in production of ROS by CYP2E1,Interactions between immune cells and microRNAs in tumor microenvironment,Metabolism of Tetrahydrocannabinol (THC)
ZBTB33,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
IDH1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
MSRA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
SLC39A14,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
TRPC3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RPL3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
MYC,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
AGPAT2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
TBC1D4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [31]:
df.to_csv(r"/Users/coribushman/Desktop/WikiPathwaysDataframe.csv", sep="\t")

In [73]:
def getInteractingProteinList(proteinName):
    
    df = pd.read_csv(r"/Users/coribushman/Desktop/WikiPathwaysDataframe.csv", sep="\t")
    pathways=[]
    try:
        row = df.loc[[proteinName]]
        
    except:
        print("Protein not found in dataframe")
        return
    
    for pathway in row:
        if row[pathway][proteinName]==True:
            pathways.append(pathway)

    genes_in_pathways = []
    for pw in pathways:
        for g in genesPerPathway[pw]:
            if g not in genes_in_pathways:
                genes_in_pathways.append(g)

    return genes_in_pathways

In [74]:
getInteractingProteinList("MYC")

Protein not found in dataframe
